In [ ]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.optim as optim
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import sqlite3
import pickle
from sklearn.utils import resample
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import GradientBoostingRegressor

# Load Data
conn = sqlite3.connect('../train_mTSP.sqlite3')
instances = pd.read_sql_query("SELECT * FROM instances", conn)
algorithms = pd.read_sql_query("SELECT * FROM algorithms", conn)
conn.close()

data = pd.merge(instances, algorithms, on="instance_id")
data = data[~((data['strategy'] == 'branch and cut') & (data['nr_cities'] > 40))]


# Normalization
for col in ['time_taken', 'distance_gap']:
    data[col + '_norm'] = (data[col] - data[col].min()) / (data[col].max() - data[col].min() + 1e-8)

# Train composite score model (GradientBoosting)
score_features = ['normalized_cost', 'time_taken_norm', 'distance_gap_norm']
X_score = data[score_features]
y_score = data['normalized_cost']
score_model = GradientBoostingRegressor(n_estimators=100, random_state=42)
score_model.fit(X_score, y_score)

# Predict composite scores using the trained regressor
data['composite_score'] = score_model.predict(X_score)

data = data[~((data['strategy'].str.lower() == 'branch and cut') & (data['nr_cities'] > 40))]

penalty_map = {
    'kmeans-greedy': 0.1,
    'ant colony': 0.00,
    'greedy': 0.00,
    'branch and cut': 0.05,
}
data['composite_score'] += data['strategy'].str.lower().map(penalty_map).fillna(0)

# Select best strategy per instance
best_strategies = data.loc[data.groupby('instance_id')['composite_score'].idxmin()]

# Select features and labels
features = best_strategies[['nr_cities', 'nr_salesmen', 'average_distance', 'stddev_distance', 'density',
                            'salesmen_ratio', 'bounding_box_area', 'aspect_ratio', 'spread',
                            'cluster_compactness', 'mst_total_length', 'entropy_distance_matrix']]
labels = pd.get_dummies(best_strategies['strategy'])
labels_order = list(labels.columns)
with open('labels_order-v2.txt', 'w') as f:
    for col in labels_order:
        f.write(col + '\n')

# Augment underrepresented classes
df = features.copy()
df['strategy'] = best_strategies['strategy'].values

major_class = df['strategy'].value_counts().idxmax()
max_count = df['strategy'].value_counts().max()

augmented = []
for strat in df['strategy'].unique():
    subset = df[df['strategy'] == strat]
    if len(subset) < max_count:
        subset_aug = resample(subset, replace=True, n_samples=max_count - len(subset), random_state=42)
        augmented.append(subset_aug)

if augmented:
    df = pd.concat([df] + augmented, ignore_index=True)

df = df.sample(frac=1, random_state=42).reset_index(drop=True)
features = df.drop(columns=['strategy'])
labels = pd.get_dummies(df['strategy'])

# SMOTE
smote = SMOTE(random_state=42, k_neighbors=3)
features_res, labels_res = smote.fit_resample(features, labels.values.argmax(axis=1))
labels_res = pd.get_dummies(labels.columns[labels_res])


# Normalize features
scaler = StandardScaler()
features_res = scaler.fit_transform(features_res)

x_train = torch.tensor(features_res, dtype=torch.float32)
y_train = torch.tensor(labels_res.values, dtype=torch.float32)

# Validation Data
conn = sqlite3.connect('../validation_mTSP.sqlite3')
val_instances = pd.read_sql_query("SELECT * FROM instances", conn)
val_algorithms = pd.read_sql_query("SELECT * FROM algorithms", conn)
conn.close()

val_data = pd.merge(val_instances, val_algorithms, on="instance_id")
val_data = val_data[~((val_data['strategy'] == 'branch and cut') & (val_data['nr_cities'] > 40))]

# Apply same normalization
for col in ['time_taken', 'distance_gap']:
    val_data[col + '_norm'] = (val_data[col] - data[col].min()) / (data[col].max() - data[col].min() + 1e-8)

# Predict composite scores using trained model
val_data['composite_score'] = score_model.predict(val_data[score_features])

val_data = val_data[~((val_data['strategy'].str.lower() == 'branch and cut') & (val_data['nr_cities'] > 40))]

# Select best strategy per validation instance
validation_best = val_data.loc[val_data.groupby('instance_id')['composite_score'].idxmin()]

validation_features = validation_best[['nr_cities', 'nr_salesmen', 'average_distance', 'stddev_distance', 'density',
                                       'salesmen_ratio', 'bounding_box_area', 'aspect_ratio', 'spread',
                                       'cluster_compactness', 'mst_total_length', 'entropy_distance_matrix']]
validation_labels = pd.get_dummies(validation_best['strategy'])

for col in labels.columns:
    if col not in validation_labels.columns:
        validation_labels[col] = 0
validation_labels = validation_labels[labels.columns].astype(float)

validation_features = scaler.transform(validation_features)
x_val = torch.tensor(validation_features, dtype=torch.float32)
y_val = torch.tensor(validation_labels.values, dtype=torch.float32)

# Model definition
class NeuralNetwork(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.BatchNorm1d(64),
            nn.LeakyReLU(),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.LeakyReLU(),
            nn.Linear(32, output_size)
        )
    def forward(self, x):
        return self.net(x)

# Focal loss definition
class FocalLoss(nn.Module):
    def __init__(self, gamma=2, weight=None):
        super().__init__()
        self.gamma = gamma
        self.weight = weight

    def forward(self, input, target):
        logpt = F.log_softmax(input, dim=1)
        pt = torch.exp(logpt)
        logpt = (1 - pt) ** self.gamma * logpt
        return F.nll_loss(logpt, target, self.weight)

# Training setup
input_size = x_train.shape[1]
output_size = y_train.shape[1]
model = NeuralNetwork(input_size, output_size)
criterion = FocalLoss(gamma=2)
optimizer = optim.Adam(model.parameters(), lr=0.0001, weight_decay=1e-2)
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=50)

best_val_loss = float('inf')
patience = 40
trigger_times = 0

# Training loop
epochs = 500
batch_size = 64
for epoch in range(epochs):
    model.train()
    permutation = torch.randperm(x_train.size(0))
    for i in range(0, x_train.size(0), batch_size):
        indices = permutation[i:i+batch_size]
        batch_X, batch_y = x_train[indices], y_train[indices]
        if batch_X.shape[0] < 2:
            continue

        optimizer.zero_grad()
        outputs = model(batch_X)
        loss = criterion(outputs, torch.argmax(batch_y, dim=1).long())
        loss.backward()
        optimizer.step()

    # Validation
    model.eval()
    with torch.no_grad():
        val_outputs = model(x_val)
        val_loss = criterion(val_outputs, torch.argmax(y_val, dim=1))

    print(f"Epoch {epoch+1}/{epochs}, Train Loss: {loss.item():.4f}, Val Loss: {val_loss.item():.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        trigger_times = 0
        torch.save(model.state_dict(), 'models_version2/best_model.pth')
    else:
        trigger_times += 1
        if trigger_times >= patience:
            print("Early stopping triggered!")
            break

    scheduler.step()

# Save scaler
with open('scaler_version2.pkl', 'wb') as f:
    pickle.dump(scaler, f)



Epoch 1/500, Train Loss: 0.5902, Val Loss: 1.1074
Epoch 2/500, Train Loss: 0.6163, Val Loss: 0.8688
Epoch 3/500, Train Loss: 0.4778, Val Loss: 0.7168
Epoch 4/500, Train Loss: 0.3739, Val Loss: 0.5959
Epoch 5/500, Train Loss: 0.4478, Val Loss: 0.5486
Epoch 6/500, Train Loss: 0.4613, Val Loss: 0.4937
Epoch 7/500, Train Loss: 0.3572, Val Loss: 0.4613
Epoch 8/500, Train Loss: 0.3583, Val Loss: 0.4325
Epoch 9/500, Train Loss: 0.1956, Val Loss: 0.4008
Epoch 10/500, Train Loss: 0.4210, Val Loss: 0.3841
Epoch 11/500, Train Loss: 0.2238, Val Loss: 0.3561
Epoch 12/500, Train Loss: 0.2022, Val Loss: 0.3175
Epoch 13/500, Train Loss: 0.2255, Val Loss: 0.3111
Epoch 14/500, Train Loss: 0.3481, Val Loss: 0.2800
Epoch 15/500, Train Loss: 0.1982, Val Loss: 0.2755
Epoch 16/500, Train Loss: 0.2293, Val Loss: 0.2626
Epoch 17/500, Train Loss: 0.2292, Val Loss: 0.2244
Epoch 18/500, Train Loss: 0.2129, Val Loss: 0.2370
Epoch 19/500, Train Loss: 0.2085, Val Loss: 0.2234
Epoch 20/500, Train Loss: 0.1366, Val Lo

In [37]:
print(best_strategies['strategy'].value_counts())

with torch.no_grad():
    train_outputs = model(x_train)
    train_predictions = torch.argmax(train_outputs, dim=1)
    print(pd.Series(train_predictions.numpy()).value_counts())

strategy
Branch and Cut    597
Greedy            459
Ant Colony         31
KMeans-Greedy       2
Name: count, dtype: int64
1    641
3    616
0    599
2    532
Name: count, dtype: int64
